<a href="https://colab.research.google.com/github/vrige/Car_classification/blob/main/Car_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.8.2


In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [27]:
# %cd /gdrive/My Drive
# new_dir = "deep_learning_selfprojects"
# if not os.path.isfile(new_dir):
#   os.mkdir(new_dir)
# print(os.listdir)
# %cd /gdrive/My Drive/deep_learning_selfprojects
# dirname = "Car_classification"
# if not os.path.isfile(dirname):
#    os.mkdir(dirname)

In [29]:
%cd /gdrive/My Drive/deep_learning_selfprojects/Car_classification

/gdrive/My Drive/deep_learning_selfprojects/Car_classification


In [32]:
pwd

'/gdrive/MyDrive/deep_learning_selfprojects/Car_classification'

In [ ]:
# !unzip archive.zip